# Model Experimentation: Hyperparameter Optimization
This notebook conducts experiments on hyperparameter optimization.

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [2]:
version = '20200504'
data_dir = "../data/"
input_file = data_dir + '{}_dataset.csv'.format(version)

images_dir = data_dir + 'images/'
indices_dir = data_dir + 'indices/'
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'

## Load Dataset

In [3]:
%%time
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (1065595, 69)
CPU times: user 14.7 s, sys: 881 ms, total: 15.6 s
Wall time: 15.1 s


,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2019-2020,ui_2019-2020,nbi_2019-2020,brba_2019-2020,nbai_2019-2020,mbi_2019-2020,baei_2019-2020,la_guajira,target,area
0,0.15100,0.13225,0.14240,0.16430,0.1899,0.25700,0.2960,0.25510,0.32295,0.0396,...,-0.438625,0.151655,0.209614,0.479385,-0.665204,-0.060444,0.948025,1,3,0
1,0.15100,0.12885,0.13790,0.16185,0.1899,0.25700,0.2960,0.25755,0.32295,0.0396,...,-0.435450,0.151655,0.213526,0.488330,-0.663010,-0.059064,0.952352,1,3,0
2,0.15895,0.13730,0.15185,0.18915,0.2264,0.28555,0.3268,0.28085,0.35740,0.0416,...,-0.447333,0.145385,0.222971,0.473118,-0.661291,-0.060496,0.911748,1,3,0


## Resample Dataset

In [4]:
data = model_utils.resample(data, num_neg_samples=20000, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (204031, 69)


,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2019-2020,ui_2019-2020,nbi_2019-2020,brba_2019-2020,nbai_2019-2020,mbi_2019-2020,baei_2019-2020,la_guajira,target,area
0,0.1748,0.12640,0.12590,0.12075,0.1288,0.2291,0.28325,0.2551,0.31165,0.03960,...,-0.325365,0.336843,0.097095,0.493299,-0.683798,-0.120547,1.149291,1,2,0
1,0.2093,0.19810,0.20230,0.21700,0.2178,0.2767,0.32240,0.3020,0.34780,0.03530,...,-0.212563,0.098564,0.242471,0.716410,-0.623340,-0.022285,1.042249,1,2,0
2,0.2062,0.20505,0.21575,0.23320,0.2083,0.2836,0.33810,0.3109,0.37400,0.03575,...,-0.235786,0.266124,0.158668,0.655587,-0.632521,-0.078219,1.101300,1,2,0


## Machine Learning Pipeline

In [5]:
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
SEED = 42

## Define Features and Target Label

In [6]:
label = 'target'
features = [column  for column in data.columns[:-2]]
data[label] = data[label].replace({2:0, 3:0})

splits = data[['area']]
X = data[features]
y = data[label]

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Data dimensions: (204031, 69)
Class distribution:
0    180000
1     24031
Name: target, dtype: int64


## Logistic Regression

In [9]:
%%time
lr =  LogisticRegression()
param_grid = {'classifier__C':[0.001, 0.01, 1.0, 5.0, 10]}
lr_results = model_utils.nested_spatial_cv(lr, X, y, splits=splits, param_grid=param_grid, verbose=2);

Test Set: Maicao
Predicted  False  True  __all__
Actual                         
False      18828  1172    20000
True         209   343      552
__all__    19037  1515    20552

               precision    recall  f1-score   support

           0       0.99      0.94      0.96     20000
           1       0.23      0.62      0.33       552

    accuracy                           0.93     20552
   macro avg       0.61      0.78      0.65     20552
weighted avg       0.97      0.93      0.95     20552

F1 Score: 0.3319
Kappa Statistics: 0.3045
Precision: 0.2264
Recall: 0.6214
Accuracy: 0.9328
ROC AUC: 0.7814

Test Set: Riohacha
Predicted  False  True  __all__
Actual                         
False      19917    83    20000
True         581  2920     3501
__all__    20498  3003    23501

               precision    recall  f1-score   support

           0       0.97      1.00      0.98     20000
           1       0.97      0.83      0.90      3501

    accuracy                           0

## LinearSVC

In [ ]:
%%time
lsvc = LinearSVC(random_state=SEED)
param_grid = {'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100, 1000]}
lsvc_results = model_utils.nested_spatial_cv(lsvc, X, y, splits=splits, param_grid=param_grid, verbose=2);

## Random Forest

In [ ]:
%%time
param_grid = {
    'classifier__n_estimators': [int(x) for x in np.linspace(start = 10, stop = 500, num = 10)],
    'classifier__max_features': ['auto', 'sqrt'],
    'classifier__max_depth': [5, 6, 7, 8, 9, 10],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False]
}
rf = RandomForestClassifier(random_state=SEED)
model_utils.nested_spatial_cv(rf, X, y, splits=splits, param_grid=param_grid, search_type='random', verbose=2);

## Naive Bayes

In [7]:
%%time
gnb = GaussianNB()
gnb_results = model_utils.nested_spatial_cv(
    gnb, X, y, splits=splits, param_grid=None, search_type=None, feature_selection=None, verbose=2
);

Test Set: Maicao
Predicted  False   True  __all__
Actual                          
False      10164   9836    20000
True           0    552      552
__all__    10164  10388    20552

               precision    recall  f1-score   support

           0       1.00      0.51      0.67     20000
           1       0.05      1.00      0.10       552

    accuracy                           0.52     20552
   macro avg       0.53      0.75      0.39     20552
weighted avg       0.97      0.52      0.66     20552

F1 Score: 0.1009
Kappa Statistics: 0.0526
Precision: 0.0531
Recall: 1.0000
Accuracy: 0.5214
ROC AUC: 0.7541

Test Set: Riohacha
Predicted  False  True  __all__
Actual                         
False      13852  6148    20000
True         386  3115     3501
__all__    14238  9263    23501

               precision    recall  f1-score   support

           0       0.97      0.69      0.81     20000
           1       0.34      0.89      0.49      3501

    accuracy                       

## Ridge Classifier

In [8]:
%%time
rc =  RidgeClassifier()
param_grid = {'classifier__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
rc_results = model_utils.nested_spatial_cv(rc, X, y, splits=splits, param_grid=param_grid, verbose=2);

Test Set: Maicao
Predicted  False  True  __all__
Actual                         
False      18957  1043    20000
True         100   452      552
__all__    19057  1495    20552

               precision    recall  f1-score   support

           0       0.99      0.95      0.97     20000
           1       0.30      0.82      0.44       552

    accuracy                           0.94     20552
   macro avg       0.65      0.88      0.71     20552
weighted avg       0.98      0.94      0.96     20552

F1 Score: 0.4416
Kappa Statistics: 0.4188
Precision: 0.3023
Recall: 0.8188
Accuracy: 0.9444
ROC AUC: 0.8833

Test Set: Riohacha
Predicted  False  True  __all__
Actual                         
False      19963    37    20000
True        1293  2208     3501
__all__    21256  2245    23501

               precision    recall  f1-score   support

           0       0.94      1.00      0.97     20000
           1       0.98      0.63      0.77      3501

    accuracy                           0